In [2]:
import pandas as pd
import numpy as np

In [3]:
# Upload repairs data
dtp = pd.read_excel('C:/Users/sgulbin/Work/_Выгрузки_и_Расчеты/accidents_4_10_2020.xlsx')
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/18/2020 12:00:00 AM', '3/18/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='4/20/2020 12:00:00 AM', '3/20/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='5/24/2020 12:00:00 AM', '3/24/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = np.where(
    dtp['Дата и время ДТП']=='7/23/2020 12:00:00 AM', '3/23/2020 12:00:00 AM', dtp['Дата и время ДТП']
)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'], dayfirst=False, errors = 'coerce')
dtp.sort_values('Дата и время ДТП', inplace=True)
dtp['Дата и время ДТП'] = pd.to_datetime(dtp['Дата и время ДТП'])
dtp = dtp[dtp['Телефон водителя'].notnull()]
dtp['Телефон водителя'] = dtp['Телефон водителя'].apply(int)
dtp = dtp[dtp['ТС'].notnull()]
dtp.reset_index(drop=True, inplace = True)
dtp['ТС'] = dtp['ТС'].str.replace(' ', '')
dtp = dtp[['Дата и время ДТП', 'Регион','Телефон водителя', 'Виновность', 'ТС', 'Модель автомобиля', 'POOL', 'Тариф','Ремонт не производился','Тип ДТП', 'Вид страхования', 'Пострадавшие','Сумма по заказ-наряду','Выставлено', 'Страховая компания сумма выплаченная']].rename(columns={
    'Дата и время ДТП': 'dt',
    'Регион': 'region',
    'Телефон водителя': 'phone',
    'Виновность': 'Responsibility',
    'ТС': 'license_plate',
    'Модель автомобиля': 'vehicle_model',
    'POOL': 'pool',
    'Тариф': 'tariff',
    'Ремонт не производился': 'no repair conducted',
    'тип ДТП': 'accident type',
    'Вид страхования': 'insurance type',
    'Пострадавшие': 'injured',
    'Сумма по заказ-наряду': 'target_amount',
    'Выставлено': 'Vystavleno',
    'Страховая компания сумма выплаченная': 'insurance paid sum'
})

In [40]:
# rides = pd.read_csv('C:/Users/sgulbin/Work/Data/ride_time_by_VIN_2019Q2-2020Q1_v2.csv')
rides = pd.read_csv('C:/Users/sgulbin/Work/Data/ride_time_by_VIN_2020Q1_v2.csv')
# Upper case VIN and license plate number for successfull merge with other df
rides.VIN = rides['VIN'].str.upper()
min_by_region = rides.groupby('rent_region_en').sum()
min_by_region

,ride_time,total_time
rent_region_en,,
Grozny,4.943308e+05,8.654013e+05
Krasnodar,2.634835e+06,4.286536e+06
Krasnoyarsk,7.588211e+05,1.341923e+06
Moscow,1.963117e+08,2.926825e+08
Nizhny Novgorod,1.580395e+06,2.589733e+06
Novosibirsk,1.230735e+06,2.051756e+06
Rostov-on-Don,8.093890e+05,1.306983e+06
Samara,1.404517e+06,2.343986e+06
St. Petersburg,1.563608e+07,2.409705e+07


In [41]:
# dtp20 = dtp.loc[(dtp.dt >= '2019-04-01 00:00:00')&(dtp.dt < '2020-04-01 00:00:00')&(dtp.pool == 'No')&\
#                 (dtp.Responsibility!='Не виновен')&(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')]
dtp20 = dtp.loc[(dtp.dt >= '2020-01-01 00:00:00')&(dtp.dt < '2020-04-01 00:00:00')&(dtp.pool == 'No')&\
                (dtp.Responsibility!='Не виновен')&(dtp.Responsibility != 'Фейк')&(dtp.Responsibility != 'Угон')]
dtp20 = dtp20.replace({'region':{'Грозный': 'Grozny','Краснодар':'Krasnodar','Красноярск':'Krasnoyarsk','Москва':'Moscow',\
                         'Нижний Новгород':'Nizhny Novgorod','Новосибирск':'Novosibirsk','Ростов-на-Дону':'Rostov-on-Don',\
                         'Самара':'Samara','Санкт-Петербург':'St. Petersburg','Уфа':'Ufa','Екатеринбург':'Yekaterinburg',\
                         'ТУЛА':'Moscow'}})
count = dtp20.groupby('region').count()
# count.to_csv('\\\\delimobil.local/Delimobil/Risk/Страхование/Убыточность 2020/accidents_by_region_2019Q2-2020Q1.csv')
count = count[['dt']]
count.columns = ['accidents']
count

,accidents
region,
Grozny,4
Krasnodar,44
Krasnoyarsk,28
Moscow,2187
Nizhny Novgorod,36
Novosibirsk,33
Rostov-on-Don,27
Samara,33
St. Petersburg,385


In [42]:
by_region = pd.merge(min_by_region, count, left_index = True, right_index = True, how = 'left')
by_region['accident_frequency_per_1000_min_ride_time'] = by_region['accidents']*1000/by_region['ride_time']
by_region = by_region.sort_values(by ='accident_frequency_per_1000_min_ride_time', ascending = True )
by_region.to_csv('\\\\delimobil.local/Delimobil/Risk/Страхование/Убыточность 2020/accident_frequency_by_region_2020Q1.csv')
by_region

,ride_time,total_time,accidents,accident_frequency_per_1000_min_ride_time
rent_region_en,,,,
Grozny,4.943308e+05,8.654013e+05,4,0.008092
Moscow,1.963117e+08,2.926825e+08,2187,0.011140
Krasnodar,2.634835e+06,4.286536e+06,44,0.016699
Yekaterinburg,3.680719e+06,6.206363e+06,79,0.021463
Nizhny Novgorod,1.580395e+06,2.589733e+06,36,0.022779
Samara,1.404517e+06,2.343986e+06,33,0.023496
St. Petersburg,1.563608e+07,2.409705e+07,385,0.024623
Ufa,1.395723e+06,2.351881e+06,37,0.026510
Novosibirsk,1.230735e+06,2.051756e+06,33,0.026813


In [18]:
ins = pd.read_excel('\\\\delimobil.local/Delimobil/Risk/Страхование/Отчет по страхованию/test/AllPolicies_Agg_2019Q2 - 2020Q1.xlsx', \
                    sheet_name='OSAGO_Prem_2019')
ins = ins[['Экспозиция', 'Заработанная премия', 'Марка/Модель ТС по ПТС', 'VIN                 ', 'ГРЗ']]
ins.columns = ['exp', 'ep', 'model', 'VIN', 'license_plates']
ins.VIN = ins['VIN'].str.upper()
ins = ins.groupby('VIN').sum()
df = pd.merge(rides, ins, on = 'VIN', how = 'left')
exp_by_region = df.groupby('rent_region_en').sum()
exp_by_region.to_csv('\\\\delimobil.local/Delimobil/Risk/Страхование/Убыточность 2020/exposure_by_region_2019Q2-2020Q1.csv')
print(exp_by_region.ride_time.sum())
exp_by_region

803097692.610003


,ride_time,total_time,exp,ep
rent_region_en,,,,
Grozny,2.481346e+06,4.363807e+06,4.436813,3.435070e+04
Krasnodar,1.141915e+07,1.921977e+07,136.758242,1.456446e+06
Krasnoyarsk,3.691316e+06,6.847826e+06,32.964286,3.232405e+05
Moscow,6.764393e+08,1.044008e+09,8203.967033,7.968389e+07
Nizhny Novgorod,8.753977e+06,1.518754e+07,98.623626,8.902343e+05
Novosibirsk,5.381901e+06,9.497048e+06,51.000000,4.381170e+05
Rostov-on-Don,4.004690e+06,7.017850e+06,43.607143,4.409579e+05
Samara,6.815816e+06,1.164030e+07,33.000000,3.156702e+05
St. Petersburg,6.018502e+07,9.912674e+07,874.348901,8.961043e+06
